In [1]:
%load_ext autoreload
%load_ext lab_black
%autoreload 2

import admix
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
import seaborn as sns
import admix_prs
import statsmodels.api as sm
import os

In [2]:
prefix = "hsq-0.25-pcausal-0.01-hermodel-gcta"
PLINK_DIR = "out/PLINK/"
TEST_BFILE = "out/PLINK/eur_test.all"

In [3]:
sim_i = 0

In [4]:
import dapgen

In [13]:
# todo: add centering option in dapgen

In [5]:
weights_dir = f"out/PRS-WEIGHTS/{prefix}"
pred_dir = f"out/PRS/{prefix}"

if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

df_weights = (
    pd.read_csv(join(weights_dir, f"sim_{sim_i}.weight.tsv.gz"), sep="\t")
    .rename(columns={"CHR": "CHROM", "A1": "ALT", "A2": "REF"})
    .set_index("SNP")
)
df_pred = admix_prs.calc_prs(TEST_BFILE, df_weights)

common snp proportion: 1.0
             CHROM  CM       POS ALT REF
snp                                     
1:752721         1   0    752721   A   G
1:754182         1   0    754182   A   G
1:760912         1   0    760912   C   T
1:768448         1   0    768448   A   G
1:779322         1   0    779322   G   A
...            ...  ..       ...  ..  ..
22:51178090     22   0  51178090   A   G
22:51181759     22   0  51181759   A   G
22:51211392     22   0  51211392   C   T
22:51212875     22   0  51212875   C   A
22:51219006     22   0  51219006   A   G

[1054151 rows x 5 columns]
             CHROM       POS ALT REF  SAMPLE_1  SAMPLE_2  SAMPLE_3  SAMPLE_4  \
SNP                                                                            
1:752721         1    752721   A   G       0.0       0.0       0.0       0.0   
1:754182         1    754182   A   G       0.0       0.0       0.0       0.0   
1:760912         1    760912   C   T       0.0       0.0       0.0       0.0   
1:768448    

admix.data.geno_mult_mat: 100%|██████████| 1030/1030 [18:37<00:00,  1.08s/it]


In [10]:
def submit_predict(prefix, sim_i):
    weights_dir = f"out/PRS-WEIGHTS/{prefix}"
    pred_dir = f"out/PRS/{prefix}"

    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)

    df_weights = (
        pd.read_csv(join(weights_dir, f"sim_{sim_i}.weight.tsv.gz"), sep="\t")
        .rename(columns={"CHR": "CHROM", "A1": "ALT", "A2": "REF"})
        .set_index("SNP")
    )
    df_pred = admix_prs.calc_prs(TEST_BFILE, df_weights)
    df_pred.to_csv(join(pred_dir, f"sim_{sim_i}.prs.tsv.gz"), sep="\t")


import submitit

executor = submitit.SgeExecutor(folder="./submitit-logs")

executor.update_parameters(
    time_min=70,
    memory_g=20,
    setup=[
        "export PATH=~/project-pasaniuc/software/miniconda3/bin:$PATH",
        "export PYTHONNOUSERSITE=True",
    ],
)

jobs = executor.map_array(submit_predict, [prefix] * 10, np.arange(10))

In [62]:
sim_i = 2

In [63]:
SIM_DIR = f"../01-simulate-pheno/out/PHENO/{prefix}"

In [64]:
# read PRS
df_prs = pd.read_csv(
    f"out/PRS-WEIGHTS/{prefix}/sim_{sim_i}.test_prs.tsv.gz", sep="\t", index_col=0
)
# read genetic value and phenotype
df_pheno_g = (
    pd.read_csv(
        join(f"{SIM_DIR}/sim.pheno_g.tsv.gz"),
        sep="\t",
        index_col=0,
    )[[f"SIM_{sim_i}"]]
    .rename(columns={f"SIM_{sim_i}": "GV"})
    .reindex(df_prs.index)
)
df_pheno = (
    pd.read_csv(join(f"{SIM_DIR}/sim.pheno.tsv.gz"), sep="\t", index_col=0)[
        [f"SIM_{sim_i}"]
    ]
    .rename(columns={f"SIM_{sim_i}": "PHENO"})
    .reindex(df_prs.index)
)
df_prs = pd.concat([df_pheno, df_pheno_g, df_prs], axis=1)

In [65]:
import seaborn as sns

In [66]:
df_pred = df_prs[[f"SAMPLE_{i}" for i in range(1, 501)]]

pred_interval = np.quantile(df_pred, q=[0.1, 0.9], axis=1)

In [67]:
intercept = (df_prs.MEAN - df_prs.GV).mean()
df_plot = pd.DataFrame(
    {
        "GV": df_prs.GV,
        "MEAN": df_prs.MEAN - intercept,
        "LOWER": pred_interval[0, :] - intercept,
        "UPPER": pred_interval[1, :] - intercept,
    }
)

In [68]:
np.mean(df_plot.GV <= df_plot.LOWER)

0.08569548253654867

In [69]:
np.mean(df_plot.GV <= df_plot.UPPER)

0.9126122314671179

In [60]:
sm.OLS(df_prs.GV, sm.add_constant(df_prs.MEAN)).fit().summary()

/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     GV   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                 3.296e+04
Date:                Sun, 21 Nov 2021   Prob (F-statistic):               0.00
Time:                        10:09:23   Log-Likelihood:                -5315.9
No. Observations:               21273   AIC:                         1.064e+04
Df Residuals:                   21271   BIC:                         1.065e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0372      0.002    -17.049      0.000      -0.041      -0.033
MEAN           0.9683      0.005    181.544      0.000       0.958       0.979
==============================================================================
Omnibus:                       14.135   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.745
Skew:                           0.042   Prob(JB):                     0.000628
Kurtosis:                       3.097   Cond. No.                         2.53
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""